<a href="https://colab.research.google.com/github/AasaiAlangaram/Kaggle-PlantPathalogy-Challenge-2020/blob/master/Model_ResNet_v1.01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Keras is a Deep Learning library for Python, that is simple, modular, and extensible.

In [0]:
import tensorflow as tf
import numpy as np
import tensorflow_hub as hub
import pathlib
import datetime
import pandas as pd

In [3]:
print(f"Tensor Flow Version: {tf.__version__}")
print(f"numpy Version: {np.version.version}")

Tensor Flow Version: 2.2.0-rc4
numpy Version: 1.18.4


In [4]:
# Raw Dataset Directory
data_dir = pathlib.Path("/content/drive/My Drive/plant-pathology-2020-fgvc7/images")
image_count = len(list(data_dir.glob('*.jpg')))


# print total no of images for all classes
print(image_count)

3651


In [0]:
BATCH_SIZE = 64             # Can be of size 2^n, but not restricted to. for the better utilization of memory
IMG_HEIGHT = 227            # input Shape required by the model
IMG_WIDTH = 227             # input Shape required by the model



# Image_Data_Path = "D:/19_Projects/05_Pathology/plant-pathology-2020-fgvc7/images"


df=pd.read_csv('/content/drive/My Drive/plant-pathology-2020-fgvc7/train.csv') #training set list
columns=["healthy", "multiple_diseases", "rust", "scab"]
df_test=pd.read_csv('/content/drive/My Drive/plant-pathology-2020-fgvc7/test.csv') #testing set list

df['image_id'] = df['image_id'].astype(str)+".jpg"
df_test['image_id'] = df_test['image_id'].astype(str)+".jpg"

In [0]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [7]:
train_data_gen = image_generator.flow_from_dataframe(dataframe=df[:1460],
                                                     directory=str(data_dir),
                                                     x_col="image_id",
                                                     y_col=columns,
                                                     seed=42,
                                                     batch_size=BATCH_SIZE,
                                                     shuffle=True,
                                                     class_mode="raw",
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH))#Resizing the raw dataset

validation_data_gen = image_generator.flow_from_dataframe(dataframe=df[1460:],
                                                     directory=str(data_dir),
                                                     x_col="image_id",
                                                     y_col=columns,
                                                     seed=42,
                                                     batch_size=BATCH_SIZE,
                                                     shuffle=True,
                                                     class_mode="raw",
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH))#Resizing the raw dataset

#test data
test_generator=image_generator.flow_from_dataframe(
    dataframe=df_test[:],
    directory=str(data_dir),
    x_col="image_id",
    batch_size=1,
    seed=42,
    shuffle=False,
    class_mode=None,
    target_size=(227,227))

Found 1460 validated image filenames.
Found 361 validated image filenames.
Found 1821 validated image filenames.


In [8]:

STEP_SIZE_TRAIN=train_data_gen.n//train_data_gen.batch_size
STEP_SIZE_VALID=validation_data_gen.n//validation_data_gen.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size



model = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/imagenet/resnet_v1_152/feature_vector/4",
                   trainable=False),  # Can be True, see below.
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(4, activation='softmax')
])
model.build([None, 227, 227, 3])  # Batch input shape.

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     multiple                  58295232  
_________________________________________________________________
batch_normalization (BatchNo multiple                  8192      
_________________________________________________________________
dense (Dense)                multiple                  8196      
Total params: 58,311,620
Trainable params: 12,292
Non-trainable params: 58,299,328
_________________________________________________________________


In [0]:
model.compile(optimizer='rmsprop', loss="categorical_crossentropy", metrics=['accuracy'])

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs={}):
        if (logs.get("accuracy")==1.00 ):
            print("\nReached 100% accuracy so stopping training")
            self.model.stop_training =True
callbacks = myCallback()

# TensorBoard.dev Visuals
log_dir="logs\\fit\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [10]:
history = model.fit(
      train_data_gen,
      steps_per_epoch=STEP_SIZE_TRAIN,
      validation_data=validation_data_gen,
      validation_steps=STEP_SIZE_VALID,
      epochs=1,
      callbacks=[tensorboard_callback])

model.save("/content/drive/My Drive/plant-pathology-2020-fgvc7/save4/")

SUB_PATH = "/content/drive/My Drive/plant-pathology-2020-fgvc7/submission_sample/sample_submission.csv"
sub = pd.read_csv(SUB_PATH)

test_generator.reset()
pred=model.predict_generator(test_generator,steps=STEP_SIZE_TEST,verbose=1)

sub.loc[:, 'healthy':] = pred
sub.to_csv('submission.csv', index=False) #submit this file
sub.head()

22/22 [==============================] - 1555s 71s/step - loss: 1.0579 - accuracy: 0.6009 - val_loss: 0.8846 - val_accuracy: 0.6531
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: /content/drive/My Drive/plant-pathology-2020-fgvc7/save4/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/plant-pathology-2020-fgvc7/save4/assets


Instructions for updating:
Please use Model.predict, which supports generators.


Instructions for updating:
Please use Model.predict, which supports generators.


1821/1821 [==============================] - 1845s 1s/step


,image_id,healthy,multiple_diseases,rust,scab
0,Test_0,0.256134,0.224033,0.175010,0.344822
1,Test_1,0.202501,0.104363,0.615629,0.077508
2,Test_2,0.438471,0.120233,0.215713,0.225582
3,Test_3,0.769471,0.033152,0.104756,0.092620
4,Test_4,0.073574,0.007314,0.915724,0.003388
